# MySQL - Google Finance

In [ ]:
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

### Get credentials

In [ ]:
with open('../_credentials/mysql_credentials.txt') as f:
    contents = f.read().split('\n')
    USER = contents[0]
    NAME = contents[1]
    HOST = contents[2]
    PWD = contents[3]

In [ ]:
!find ../_data | grep portfolio.txt

In [ ]:
file = "../_data/portfolio.txt"
DB = "Googl_db"

In [ ]:
df = pd.read_csv(file, sep=':')
df.head()

## Scrape Google Finance

### Connect to MySql

In [ ]:
def connect_cursor(host, user, password, db_name):
    conx = pymysql.connect(host=host, user=user, password=password, db=db_name,
                           charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    return conx, conx.cursor()

In [ ]:
conx, cursor = connect_cursor(HOST, USER, PWD, DB)

In [ ]:
cursor.execute('show tables;')
cursor.fetchall()

### Text file to MySql tables

In [ ]:
with open(file, 'r') as f:
    line_count = 0
    stocks_set = set()
    
    for line in f:
        line = line.strip()

        if line_count == 0:
            headers = line.split(':')
            headers = [x.replace(' ','_') for x in headers]
            
            cursor.execute("DROP TABLE IF EXISTS stocks;")
            query = """CREATE TABLE IF NOT EXISTS stocks ({} VARCHAR(10), {} VARCHAR(30));
            """.format(headers[0], headers[1])
            cursor.execute(query)
            
            cursor.execute("DROP TABLE IF EXISTS holdings;")
            query = """CREATE TABLE IF NOT EXISTS holdings ({} VARCHAR(10), {} DECIMAL(10,2), {} INT, {} DATE);
            """.format(headers[0], headers[2], headers[3], headers[4])
            cursor.execute(query)
            
            line_count += 1
            continue  
            
        data = line.split(':')
        
        query = """INSERT INTO holdings VALUES ("{}", "{}", "{}", "{}");
        """.format(data[0], data[2], data[3], data[4])
        cursor.execute(query)
        
        stock_info = (data[0], data[1])
        stocks_set.add(stock_info)

In [ ]:
for stock_info in stocks_set:
    stock_query = """INSERT INTO stocks VALUES ("{}", "{}");""".format(stock_info[0], stock_info[1])
    cursor.execute(stock_query)

In [ ]:
conx.commit()
conx.close()

In [ ]:
conx, cursor = connect_cursor(HOST, USER, PWD, DB)

In [ ]:
cursor.execute('select * from holdings')
cursor.fetchall()

### Create table

In [ ]:
conx, cursor = connect_cursor(HOST, USER, PWD, DB)

In [ ]:
create_table = '''CREATE TABLE IF NOT EXISTS `users3` (
    `id` int(11) NOT NULL AUTO_INCREMENT,
    `email` varchar(255) COLLATE utf8_bin NOT NULL,
    `password` varchar(255) COLLATE utf8_bin NOT NULL,
    PRIMARY KEY (`id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1;'''

In [ ]:
create_table2 = '''CREATE TABLE IF NOT EXISTS {TABLE} (
    {ID} VARCHAR(255),
    {FIELD1} VARCHAR(255),
    {FIELD2} VARCHAR(255));'''.format(TABLE='users2', ID='id', FIELD1='email', FIELD2='password')

In [ ]:
with open(file, 'r') as f:
    line_count = 0
    stocks_set = set()
    for line in f:
        line = line.strip()

        if line_count == 0:
            headers = line.split(':')
            headers = [x.replace(' ','_') for x in headers]
            cursor.execute("DROP TABLE IF EXISTS stocks;")
            cursor.execute("DROP TABLE IF EXISTS holdings;")
            
            query1 = """CREATE TABLE IF NOT EXISTS stocks ({} VARCHAR(10), {} VARCHAR(30));
            """.format(headers[0], headers[1])
            query2 = """CREATE TABLE IF NOT EXISTS holdings ({} VARCHAR(10), {} DECIMAL(10,2), {} INT, {} DATE);
            """.format(headers[0], headers[2], headers[3], headers[4])
            print(query2)
            cursor.execute(query1)
            cursor.execute(query2)
            
            line_count += 1
            continue        
        data = line.split(':')
        stock_info = (data[0], data[1])
        stocks_set.add(stock_info)
        
        holdings_query = """INSERT INTO holdings VALUES ("{}", "{}", "{}", "{}");""".format(data[0], data[2], data[3], data[4])
        cursor.execute(holdings_query)
        
for s_info in stocks_set:
    stock_query = """INSERT INTO stocks VALUES ("{}", "{}");""".format(s_info[0], s_info[1])
    cursor.execute(stock_query)
    
conx.commit()
conx.close()

### DataFrame to MySql table

In [ ]:
credx = "mysql+pymysql://{user}:{pw}@{host}/{db}?charset=utf8".format(user=USER, host=HOST, pw=PWD, db=DB)
engine = create_engine(credx, pool_recycle=1, pool_timeout=57600, echo=False)
conx = engine.connect()

In [ ]:
df.to_sql(con=conx, name='university_dataset_ca2', if_exists='append', chunksize=100)

In [ ]:
conx, cursor = connect_cursor(HOST, USER, PWD, DB)

In [ ]:
cursor.execute('show tables;')
cursor.fetchall()

In [ ]:
conx.close()